# Cvičenie 5: Metodológia trénovania a vyhodnotenia neurónových sietí

Cieľom tohto cvičenia je oboznámiť vás so základnými krokmi, ktoré by ste mali vykonať pri trénovaní neurónových sietí. Tieto kroky platia všeobecne pre všetky metódy umelej inteligencie, ale niektoré úlohy sú špecifické pre neurónové siete. Metodológiu vysvetlíme na klasifikačnej úlohe, na definíciu a trénovanie neurónovej siete budeme používať knižnicu [Keras](https://keras.io). Ak Keras alebo tensorflow ešte nemáte nainštalovaný, nainštalujte si ho podľa [tohto návodu](https://github.com/ianmagyar/neural-networks-course/blob/master/labs/lab00-getting-started.md).

Postup vývoja neurónových sietí vieme rozdeliť do nasledujúcich krokov:
1. predspracovanie údajov
2. návrh siete
3. trénovanie siete
4. vyhodnotenie siete

## 1. Predspracovanie údajov

Predspracovanie údajov je prvý krok, ktorý v sebe zahŕňa hneď niekoľko úloh:
* načítanie datasetu
* výber príznakov
* normalizácia hodnôt
* vektorizácia vstupov a výstupov
* rozdelenie datasetu na trénovaciu, testovaciu a validačnú množinu.

Postup pri predspracovaní údajov ukážeme na Iris datasete. Stiahnite si [dataset s implementáciou neurónovej siete](sources/lab05/lab5.zip).

### 1.1. Načítanie datasetu

Na načítanie datasetu existujú rôzne knižnice pre Python, jedna populárna z nich je knižnica `pandas`. Knižnica dokáže načítať rôzne formátované dáta, napríklad formáty csv, html, json, hdf5 a SQL. Náš dataset vieme načítať priamo zo súboru csv nasledovným spôsobom:

In [ ]:
import pandas as pd
dataset = pd.read_csv('iris.csv')
print(dataset)

Načítaný dataset má typ DataFrame. K ľubovoľným stĺpcom sa dostaneme zadaním názvu stĺpca ako index datasetu. Ak chceme zobraziť viac stĺpcov, index musí byť zoznam s názvami týchto stĺpcov.

In [ ]:
# select only column SepalLengthCm
print(dataset['SepalLengthCm'])

# select columns SepalLengthCm and SepalWidthCm
print(dataset[['SepalLengthCm', 'SepalWidthCm']])

Alternatívne vieme zobraziť stĺpce ako keby boli parametrom objektu dataset, alebo vieme použiť aj poradové číslo stĺpca (znak `:` pred čiarkou vyjadruje všetky riadky).

In [ ]:
print(dataset.SepalLengthCm)
print(dataset.iloc[:, 0])

K riadkom pristupujeme cez číselné indexy, pričom dokopy ich môžeme mať až tri. Prvé číslo vyjadruje poradové číslo prvého riadku, druhé číslo poradové číslo posledného riadku (vľavo uzavretý interval, podľa [pravidiel indexovania v Pythone](https://www.digitalocean.com/community/tutorials/how-to-index-and-slice-strings-in-python-3)), a tretie číslo step. Takto vieme napríklad vypísať každý druhý riadok z intervalu 1-10:

In [ ]:
print(dataset[0:10:2])

Alternatívne môžete použiť aj `loc` funkciu DataFrame-ov (podľa indexu atribútu; druhý index vyjadruje otvorený interval), alebo `iloc` funkciu (podľa poradia; druhý index vyjadruje otvorený interval)

In [ ]:
print(dataset.loc[0:9:2])
print(dataset.iloc[0:9:2])

Indexovanie riadkov a stĺpcov viete aj kombinovať, na poradí nezáleží:

In [ ]:
print(dataset[:10:2]['SepalLengthCm'])
print(dataset['SepalLengthCm'][:10:2])

Z datasetu viete vybrať iba niektoré riadky aj na základe hodnoty niektorého atribútu použitím `lambda` funkcie. Napríklad, pre všetky riadky, kde hodnota SepalLengthCm je viac ako 5:

In [ ]:
print(dataset.loc[lambda df:df.SepalLengthCm > 5, :])

Všetky tieto podmnožiny majú typ `DataFrame`. Ak chcete hodnoty použiť ako zoznam, resp. zoznam zoznamov, musíte pridať `values`:

In [ ]:
dataset['SepalLengthCm'][:10:].values

### 1.2. Výber príznakov

Pred výberom príznakov potrebujeme získať intuitívne pochopenie datasetu a vzťahov medzi jednotlivými atribútmi a výsledkom klasifikácie. V tomto nám pomôže knižnica `Seaborn`, ktorá slúži na vizualizáciu údajov a využíva knižnicu `matplotlib`.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# set plot style
sns.set(style="ticks")
sns.set_palette("husl")

# create plots over all dataset; for subset use iloc indexing
sns.pairplot(dataset, hue="Species")

# display plots using matplotlib
plt.show()

Uvedený kód namapuje záznamy z datasetu v každom možnom príznakovom priestore vo dvojiciach príznakov.

![Vizualizácia datasetu](sources/lab05/5.1-dataset-visualization.png)

Z grafov vidíme, že ani jedna kombinácia nám nedá lineárne separovateľný dataset, budeme teda používať všetky príznaky, ktoré vyberieme pomocou knižnice `pandas`:

In [ ]:
# split data into input (X - select the first four columns) and output (y - select last column)
X = dataset.iloc[:, :4].values
y = dataset.iloc[:, -1].values

### 1.3. Normalizácia hodnôt

Normalizácia hodnôt sa používa najmä pre zložitejšie datasety a urýchli proces trénovania neurónových sietí, ktoré lepšie pracujú s dátami z istého intervalu. Počas normalizácie sa číselné hodnoty namapujú zvyčajne na interval 0 až 1.

Neskôr boli vyvinuté špeciálne vrstvy neurónovej siete práve pre normalizáciu, dnes sa skôr používa takto automatizovaný spôsob normalizácie.

### 1.4. Vektorizácia vstupov a výstupov

Kým neurónové siete dokážu spracovať iba číselné hodnoty, skoro všetky datasety obsahujú aj nečíselné údaje (reťazce, kategórie, booleovské hodnoty, atď.). Preto je potrebné, aby sme tieto hodnoty premenili na vektorovú reprezentáciu. Pri vektorizácii upravíme výstupy na formu *n* čísel, kde *n* je počet tried pri klasifikácii. Každý vektor bude obsahovať práve jednu 1 a ostatné hodnoty budú 0, tieto čísla vyjadrujú mieru príslušnosti k jednotlivým triedam.

V našom datasete potrebujeme upraviť očakávaný výstup, ktorý zatiaľ má formu reťazca. Pri vektorizácii vieme využiť `LabelEncoder` z knižnice `scikit-learn`:

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
# transform string labels into number values 0, 1, 2
y1 = encoder.fit_transform(y)

# transform number values into vector representation
Y = pd.get_dummies(y1).values

**Poznámka:** v niektorých prípadoch dokážete reťazce nahradiť jednoduchými číslami, takýto spôsob ale predpokladá, že čísla, ktoré sú blízko sebe vyjadrujú koncepty, ktoré sú veľmi podobné. Napríklad, ak máme stĺpec s hodnotami *low*, *middle*, *high*, tieto hodnoty vieme nahradiť číslami 1, 2 a 3. Rovnaký spôsob ale nemôžeme použiť s hodnotami ako napríklad značky auta: *Škoda* (1), *Audi* (2), *Lada* (3), pretože neurónová sieť by predpokladala, že Lada (3) je viac podobná Audi (2) ako Škodovke (1).

### 1.5. Rozdelenie datasetu na trénovaciu, testovaciu a validačnú množinu

Ďalšou úlohou je rozdelenie množiny na trénovaciu a testovaciu. Na to použijeme ďalšiu funkciu z knižnice `scikit-learn`, a to `train_test_split` ([dokumentácia](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)), ktorá zachová poradie vstupov a výstupov a má tri dôležité parametre:
1. zoznam vstupov
2. zoznam výstupov
3. test_size - veľkosť testovacej množiny medzi 0 a 1 (môžete použiť aj train_size)

Pre opakovateľnosť trénovania je odporúčané používať random seed zadaním parametra `random_state`.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

Validačná množina sa pri jednoduchých datasetoch až tak často nepoužíva, slúži ako testovacia množina počas fázy trénovania a môže byť použitá ako podmienka pre ukončenie trénovania.

## 2. Návrh siete

Na definíciu siete použijeme knižnicu Keras, v ktorej potrebujem tri veci na vytvorenie jednoduchej siete:

1. model - v tomto kroku použijeme jednoduchý feed-forward sekvenčný model ([dokumentácia](https://keras.io/models/sequential/))
2. vrstvy - použijeme iba plne prepojené dense vrstvy ([dokumentácia](https://keras.io/layers/core/#dense))
3. optimalizátor - algoritmus, ktorý nám zadefinuje spôsob trénovania siete; my použijeme optimalizátor Adam ([dokumentácia](https://keras.io/optimizers/#adam))

V Kerase musíme najprv vytvoriť model a doňho pridať vrstvy. Pri definícii vrstiev potrebujeme zadať počet neurónov vo vrstve a aktivačnú funkciu. Špecifická je prvá vrstva v sieti, keďže pre ňu potrebujeme zadefinovať aj počet vstupných neurónov. Počet neurónov v poslednej vrstve má zodpovedať formátu výstupu siete.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

model = Sequential()

model.add(Dense(10,input_shape=(4,),activation='tanh'))
# TODO: add dense layer with 8 neurons and tanh activation function
model.add(Dense(8, activation='tanh'))
# TODO: add dense layer with 6 neurons and tanh activation function
model.add(Dense(6, activation='tanh'))
# TODO: add dense layer with 3 neurons and softmax activation function
model.add(Dense(3, activation='softmax'))

Pred trénovaním siete ešte potrebujeme zadefinovať spôsob trénovania cez optimalizátor. Urobíme tak zavolaním funkcie `compile` ([dokumentácia](https://keras.io/models/sequential/#compile)) s nasledujúcimi parametrami:
* optimizer (optimalizátor)
* loss function (chybová funkcia)
* metrics (metriky) - nie je povinné, slúži iba na vyhodnotenie neurónovej siete

In [ ]:
model.compile(Adam(lr=0.04), 'categorical_crossentropy', metrics=['accuracy'])

## 3. Trénovanie siete

Ak sme spokojní so sieťou, môžeme ju začať trénovať pomocou fukcie `fit` ([dokumentácia](https://keras.io/models/sequential/#fit)). V tomto kroku použijeme iba niektoré parametre funkcie:
* vstupy - pole vstupov z trénovacej množiny
* výstupy - pole so správnymi výstupmi z trénovacej množiny
* počet epoch - epocha je jedna iterácia nad celou trénovacou množinou

Ďalšie často používané parametre:
* batch_size - batch slúži na presnejšiu gradient aktualizáciu
* verbose - definuje, či chceme zobraziť progress počas trénovania; môže mať hodnoty 0 (žiaden výstup), 1 (medzivýsledok po každom batchi) a 2 (medzivýsledok po epoche)

In [ ]:
model.fit(X_train, y_train, epochs=100)

## 4. Vyhodnotenie siete

Vyhodnotenie siete pozostáva z dvoch základných úloh: testovanie a vyhodnotenie. Pre testovanie musíme získať výstupy ku vstupným hodnotám z trénovacej množiny pomocou metódy `predict` ([dokumentácia](https://keras.io/models/sequential/#predict)):

In [ ]:
y_pred = model.predict(X_test)

Ďalej porovnáme ozajstné výstupy s očakávanými. Keďže výstup má vektorovú reprezentáciu, potrebujeme zistiť pozíciu kde sa nachádza najväčšia hodnota vo vektore. V tomto nám pomôže knižnica `numpy`, ktorú sme zatiaľ nepoužili explicitne, ale podporuje všetky už použité knižnice. Jedná sa o efektívne a optimalizované riešenie práce s poľami.

Pre vyhodnotenie našej siete použijeme konfúznu maticu. Konfúzna matica je tabuľková reprezentácia, kde v riadkoch máme očakávané triedy a v stĺpcoch vypočítané (predikované). V bunkách tabuľky sú uložené počty príkladov klasifikované v danej kombinácii očakávanej a predikovanej triedy. Ideálny klasifikátor bude mať všetky hodnoty po hlavnej diagonále (ďalšie informácie nájdete na [wikipédii](https://en.wikipedia.org/wiki/Confusion_matrix)).

In [ ]:
import numpy as np

y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test_class, y_pred_class))

Z konfúznej matici potom vieme vypočítať ďalšie metriky, ako presnosť (accuracy), návratnosť (recall) a precizita (precision):

In [ ]:
print(classification_report(y_test_class, y_pred_class))

Presnosť popisuje samotný klasifikátor a vypočíta sa nasledovne:

$ACC = \frac{TP + TN}{P + N}$

kde TP + TN je suma správne klasifikovaných príkladov (na hlavnej diagonále) a P + N je počet všetkých príkladov.

Návratnosť a precizita popisujú klasifikátor pre danú triedu, vypočítajú sa nasledovne:

$REC = \frac{TP}{P}$

$PREC = \frac{TP}{TP + FP}$

kde TP je počet správne klasifikovaných príkladov z danej triedy, P je počet príkadov z danej triedy v testovacej množine a FP je počet príkladov z testovacej množiny nesprávne klasifikovaných do tejto triedy.

Metóda `classification_report` vypočíta ešte hodnotu F1, ktorá je harmonický priemer návratnosti a precizity:

$F1 = 2 \cdot \frac{REC \cdot PREC}{REC + PREC}$